# Logic Development for Mobile App Combatting Lottery Addiction

In this project, we'll be developing the logic for a mobile app that would be built and used for helping folks overcome gambling addiction, particularly focusing on the lottery.

We'll be using the 6/49 lottery (see below for more info) to provide users answers to the following questions:

- What is the probability of winning the big prize?
- What is the probability of winning the big prize if we play X number of tickets?
- What is the probability of having at least X winning numbers on a single ticket?

More info on 6/49 lottery: https://en.wikipedia.org/wiki/Lotto_6/49

In addition to developing the above scenarios to show folks the probability of meeting the criteria above, we'll also be looking at historical data from the 6/49 lottery provided by kaggle. A link to the data is below:
> https://www.kaggle.com/datascienceai/lottery-dataset

## Quick Note about the 6/49 Lottery

In the 6/49 lottery, 6 numbers are drawn from a set of 49 numbers that range from 1 to 49. A key component of this drawing is that it is done *without replacement*, meaning when one number is drawn, it is not put back into the set to be potentially chosen again. Also, the order of the numbers on the ticket do not matter for winning.

## Helper Functions to develop

Now, in order to answer the above questions, we,'ll need to create some functions that help us easily calculate **factorials** and the number of **combinations** (since, as stated above, the order does not matter for winning).

In [2]:
# Create the factorial function
# n is the number of objects in the set
def factorial(n):
    
    # we'll start at 1 because if we start at 0, as you'll see below, we'll keep multiplying...
    # ...by 0, which would just give us 0
    result = 1
    
    # as factorials go, we'll start at n, then go down by 1 until we get to 1
    for number in range(n, 1, -1):
        result = result * number
    return result

# test out the factorial function. The below should give us 120
print(factorial(5))

120


In [3]:
# Create the combinations function using the factorial function to help us
# n is the number of objects in the set
# k is the number of objects that we take out/care about
def combinations(n, k):
    permutations = factorial(n) / factorial(n - k)
    result = permutations / factorial(k)
    return result

# test out the combinations function. It should result in 2203961430
print(combinations(34, 18))

2203961430.0


# Function for our first question - what is the probability of winning the big prize?

The first function we'll develop to be used by the app is a function that provides the probability of hitting the jackpot. We want this function to do the following:

- Take in 6 unique numbers as inputs from 1 to 49
- Bundle these six numbers into a Python list as an input for our function
- Print the probability value in a friendly, readable way for our users

In [4]:
# create a function with the above specifications
def one_ticket_probability(list_of_six):
    
    # calculate total number of combinations for 6/49 lottery
    n_combinations = combinations(49, 6)
    
    # calculate the probability of getting the one right combination
    probability_winning = (1 / n_combinations) * 100
    
    # create message to show user
    message = """The probability of winning for that combination 
    (and every combination you put in for that matter") is {:.10f}%"""
    message_with_probability = message.format(probability_winning)
    
    # return the message
    return message_with_probability
    

# test function
print(one_ticket_probability([1,2,3,4,5,6]))
print(one_ticket_probability([28,29,30,31,32,33]))

The probability of winning for that combination 
    (and every combination you put in for that matter") is 0.0000071511%
The probability of winning for that combination 
    (and every combination you put in for that matter") is 0.0000071511%


# Comparing ticket to historical wins

We also want to provide users the ability to check if their ticket had won in the past. We'll use kaggle's dataset mentioned above to check whether the numbers selected had won in the past.

In [5]:
# import pandas
import pandas as pd

# read in the dataset
historical_649 = pd.read_csv("649.csv")

# let's look at the first five rows of the dataset
historical_649.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


In [6]:
# let's look at the last 5 rows of the dataset
historical_649.tail()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3660,649,3587,0,6/6/2018,10,15,23,38,40,41,35
3661,649,3588,0,6/9/2018,19,25,31,36,46,47,26
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


In [7]:
# check the size of the dataset
historical_649.shape

(3665, 11)

# Function for seeing historical wins for numbers selected

With our historical wins dataset read into a dataframe, let's now create a function that does the following:

- Takes in 6 unique numbers as inputs from 1 to 49
- Bundles these six numbers into a Python list as an input for our function
- Checks how many times the combination occurred in the past winners
- Shows the probability of the combination winning the next round

We'll create two functions - one for extracting the winning sets of numbers, and the other checking how many times an inputted list of numbers matches a previous win along with providing the probability of winning the next round.

In [8]:
# first, we'll create a function for extracting the winning numbers and putting them in a Python set
def extract_numbers(row):
    number_list = [row["NUMBER DRAWN 1"],row["NUMBER DRAWN 2"],row["NUMBER DRAWN 3"],
                   row["NUMBER DRAWN 4"],row["NUMBER DRAWN 5"],row["NUMBER DRAWN 6"]]
    return set(number_list)

# test the function
print(extract_numbers(historical_649.iloc[0]))

{3, 41, 11, 12, 43, 14}


In [9]:
# with that working above, let's get our winning_number_sets
winning_number_sets = historical_649.apply(extract_numbers,axis=1)

# check that it worked
print(winning_number_sets)

0        {3, 41, 11, 12, 43, 14}
1        {33, 36, 37, 39, 8, 41}
2         {1, 6, 39, 23, 24, 27}
3         {3, 9, 10, 43, 13, 20}
4        {34, 5, 14, 47, 21, 31}
5        {8, 41, 20, 21, 25, 31}
6       {33, 36, 42, 18, 25, 28}
7        {7, 40, 16, 17, 48, 31}
8        {37, 5, 38, 10, 23, 27}
9        {4, 37, 46, 15, 48, 30}
10        {33, 38, 7, 9, 42, 21}
11      {36, 11, 43, 17, 19, 20}
12       {37, 7, 14, 47, 17, 20}
13      {35, 44, 25, 28, 29, 30}
14       {36, 39, 8, 41, 47, 18}
15       {9, 12, 13, 14, 44, 48}
16       {4, 40, 43, 44, 14, 18}
17      {34, 35, 36, 13, 16, 18}
18      {36, 11, 23, 25, 28, 29}
19       {37, 7, 45, 18, 23, 25}
20      {37, 11, 45, 18, 19, 31}
21       {8, 14, 16, 48, 18, 31}
22       {4, 11, 45, 23, 24, 25}
23        {33, 34, 3, 4, 48, 19}
24       {5, 43, 17, 21, 28, 30}
25       {36, 6, 38, 46, 17, 24}
26        {4, 9, 10, 11, 43, 46}
27       {32, 33, 7, 13, 45, 23}
28      {35, 37, 11, 18, 22, 28}
29      {35, 45, 48, 25, 26, 31}
          

In [10]:
# now we'll create a function checking to see if an inputted list of numbers matches any one of the sets above
def check_historical_occurence(user_inputted_numbers,winning_sets):
    # make the user_inputted_numbers list a set
    set_user_inputted_numbers = set(user_inputted_numbers)
    
    # check if the set exists in the winning_sets and if so, how many times
    total = winning_sets[winning_sets == set_user_inputted_numbers].shape[0]
    
    # we'll also add the probability of winning 6/49 with the numbers chosen
    n_combinations = combinations(49, 6)
    probability_winning = 1 / n_combinations * 100
    
    # now we'll create a message to print out to the user
    message1 = "The amount of times that your combination won in the past is " + str(total) + "."
    message2 = "You still have a {:.10f}% chance of winning though :(".format(probability_winning)
    final_message = message1 + "\n" + message2
    
    # return the total
    return final_message

# test the function
print(check_historical_occurence([3,41,11,12,14,43],winning_number_sets))

The amount of times that your combination won in the past is 1.
You still have a 0.0000071511% chance of winning though :(


In [11]:
# in order to test to make sure that sets that appear multiple times are winners...
# let's add the test set we used into the series  to make sure we capture all of the times it appears
test_winning_number_sets = winning_number_sets
additional_row = pd.Series([{3,41,11,12,14,43}])
new_test = test_winning_number_sets.append(additional_row)

# test the function again
print(check_historical_occurence([3,41,11,12,14,43],new_test))

The amount of times that your combination won in the past is 2.
You still have a 0.0000071511% chance of winning though :(


# What if I buy multiple tickets? - A function that will provide probability of winning in this case

Buying more tickets may lead one to believe that they are significantly increasing their odds of winning, but we'll create a function that shows what our probability really is based on the number of tickets we put in.

Our function will do the following:
- Take in the number of tickets that the user is planning to purchase
- Show the probability of winning the big prize based on the number of tickets bought

In [12]:
# create the function mentioned above
def multi_ticket_probability(number_of_tickets):
    n_combinations = combinations(49,6)
    probability_winning = number_of_tickets / n_combinations * 100
    message = "Even if you bought " + str(number_of_tickets) + " ticket(s), your chance of winning is only {:.10f}%".format(probability_winning)
    return message

# test the function
print(multi_ticket_probability(1))

Even if you bought 1 ticket(s), your chance of winning is only 0.0000071511%


# Probability of getting two, three, four, or five numbers right function

In addition to hitting the jackpot with having all six numbers, participants can also earn prizes for having at least two numbers that match the winning numbers. The more numbers that you have matching, the bigger the prize. 

To determine the probability of winning in one of these categories, we'll create a function that will do the following:
- Take in six numbers that the participant chooses
- Take in an integer between two and five that represents the number of winning numbers expected
- Provide the probability of winning based on the number of winning numbers expected

In [47]:
# create the above function
def probability_less_six(winning_numbers_expected):
    
    # playing 6/49 lottery rules
    total_numbers_in_set = 6
    total_numbers_to_choose_from = 49
    
    # come up with the number of winning outcomes
    n_combinations_expected = combinations(total_numbers_in_set,winning_numbers_expected)
    n_combinations_remaining = combinations(total_numbers_to_choose_from - winning_numbers_expected,
                                           total_numbers_in_set - winning_numbers_expected)
    n_winning_outcomes = n_combinations_remaining * n_combinations_expected
    
    # find the probability of winning based on the number of outcomes
    total_outcomes = combinations(49,6)
    probability_winning_subset = n_winning_outcomes / total_outcomes * 100
    
    # message to provide the user
    message = "If you are going for " + str(winning_numbers_expected) + " matching numbers, your probability of winning would be {:.10f}%".format(probability_winning_subset)

    return message

# test the above function
print("6:",probability_less_six(6))
print("5:",probability_less_six(5))
print("2:",probability_less_six(2))

6: If you are going for 6 matching numbers, your probability of winning would be 0.0000071511%
5: If you are going for 5 matching numbers, your probability of winning would be 0.0018878967%
2: If you are going for 2 matching numbers, your probability of winning would be 19.1326530612%


# Conclusion

In this project, we created the following four functions that could be used for a mobile app to help combat an addiction to the 6/49 lottery by giving folks a quick reference of just what their probability of winning is exactly.
- one_ticket_probability: gives the user the probability of hitting the jackpot of the 6/49 lottery
- check_historical_occurence: looks up historical winnings and tells user how many times their inputted number set won in the past and then provides the probability of that set winning again. Regardless of past performance, the probability is still the same as the output in the one_ticket_probability function above.
- multi_ticket_probability: tells the user their probability of winning based on how many tickets they buy.
- probability_less_six: since the 6/49 lottery offers smaller prizes to those who get at least two of the correct numbers, this function gives the user the probability of winning a smaller prize for getting two or up to five correct numbers instead of all six.

Thank you for reading, and don't play the lottery!